In [3]:
# 05_genai_recommendation_generation.ipynb

import os
import pandas as pd
from langchain_core.prompts import PromptTemplate     # ✅ updated import
from langchain.chains.llm import LLMChain             # ✅ updated import path for v1.x
from langchain_openai import ChatOpenAI               # ✅ OpenAI interface

# -------------------------------------------------------------------------
# 🔑 Setup OpenAI API Key
# -------------------------------------------------------------------------
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # your actual key

# -------------------------------------------------------------------------
# 🧠 Load Campaign Data Summary
# -------------------------------------------------------------------------
df = pd.read_csv("../data/adtech_campaign_performance.csv", parse_dates=["date"])

# Aggregate metrics by region and device
summary = (
    df.groupby(["region", "device"])
    .agg({
        "impressions": "sum",
        "clicks": "sum",
        "conversions": "sum",
        "cost": "sum"
    })
    .reset_index()
)
summary["CTR"] = (summary["clicks"] / summary["impressions"]) * 100
summary["CVR"] = (summary["conversions"] / summary["clicks"].replace(0, pd.NA)) * 100
summary["CPA"] = summary["cost"] / summary["conversions"].replace(0, pd.NA)

print("✅ Campaign performance summary loaded.\n")
print(summary.head())

# -------------------------------------------------------------------------
# 🧩 Define Prompt Template
# -------------------------------------------------------------------------
prompt_template = PromptTemplate(
    input_variables=["campaign_data"],
    template=(
        "You are a marketing AI assistant for AdTech campaign optimization.\n"
        "Here is the campaign performance summary:\n"
        "{campaign_data}\n\n"
        "Analyze and provide:\n"
        "1. Key insights and performance trends.\n"
        "2. Optimization recommendations.\n"
        "3. Suggested A/B testing strategies.\n"
        "4. Highlight poor-performing segments.\n\n"
        "Respond clearly with bullet points."
    )
)

# -------------------------------------------------------------------------
# 🤖 Initialize LLM (GPT-4 or GPT-3.5-turbo)
# -------------------------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4, max_tokens=700)
chain = LLMChain(prompt=prompt_template, llm=llm)

# -------------------------------------------------------------------------
# 🚀 Generate Recommendations
# -------------------------------------------------------------------------
sample_summary = summary.head(10).to_string(index=False)
response = chain.invoke({"campaign_data": sample_summary})  # ✅ Updated invoke syntax

print("\n💡 AI-Generated Optimization Insights & Recommendations:\n")
print(response["text"])


ModuleNotFoundError: No module named 'langchain.chains'